# **Proyek Analisis Sentimen: (Bagian 1) Scraping Data Ulasan Aplikasi Mamikos**

- **Nama:** Muhammad Husain Fadhlillah
- **Email Student:** mc006d5y2343@student.devacademy.id
- **Cohort ID:** MC006D5Y2343

Notebook ini bertujuan untuk melakukan scraping data ulasan (reviews) dari aplikasi Mamikos di Google Play Store.

### **1. Import Library**
Mengimpor pustaka yang diperlukan untuk proses scraping dan manipulasi data.
- `google_play_scraper`: Untuk berinteraksi dengan Google Play Store.
- `pandas`: Untuk mengelola data dalam format DataFrame.
- `time`: Untuk memberikan jeda antar request agar tidak membebani server.

In [1]:
# Instalasi library google-play-scraper
%pip install google-play-scraper pandas

# Import library yang diperlukan
import pandas as pd
import numpy as np
import os
import time
from google_play_scraper import reviews, Sort
from datetime import datetime

# Untuk mengabaikan peringatan
import warnings
warnings.filterwarnings('ignore')

print("Library berhasil diimport.")

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Library berhasil diimport.


### **2. Penentuan Parameter Scraping**
Mendefinisikan parameter utama untuk proses scraping, seperti ID aplikasi, target jumlah ulasan per skor, dan bahasa.

In [2]:
# ID aplikasi Mamikos di Google Play Store
app_id = 'com.git.mami.kos'

# Target jumlah ulasan untuk mencapai 10.000+ total
target_per_score = 3000 

# Menyiapkan list untuk menampung semua hasil scraping
all_reviews = []

print(f"Target aplikasi: {app_id}")
print(f"Target ulasan per skor: {target_per_score} ulasan")
print(f"Target total minimum: {target_per_score * 5} ulasan")

Target aplikasi: com.git.mami.kos
Target ulasan per skor: 3000 ulasan
Target total minimum: 15000 ulasan


### **3. Proses Scraping Data Secara Seimbang**
Memulai proses scraping dengan melakukan iterasi untuk setiap skor bintang (dari 1 hingga 5). Untuk setiap skor, mengambil ulasan sebanyak target yang telah ditentukan.

In [3]:
def scrape_reviews_with_retry(app_id, score, target_count, max_retries=3):
    """
    Fungsi untuk scraping dengan retry mechanism
    """
    for attempt in range(max_retries):
        try:
            print(f"Percobaan ke-{attempt + 1} untuk skor {score}")
            
            result, token = reviews(
                app_id,
                lang='id',                # Bahasa: Indonesia
                country='id',             # Negara: Indonesia
                count=target_count,       # Jumlah ulasan yang ingin diambil
                filter_score_with=score   # Filter berdasarkan skor bintang
            )
            
            print(f"✅ Berhasil mengambil {len(result)} ulasan untuk skor {score}")
            return result
            
        except Exception as e:
            print(f"❌ Percobaan ke-{attempt + 1} gagal untuk skor {score}: {e}")
            if attempt < max_retries - 1:
                wait_time = (attempt + 1) * 5
                print(f"Menunggu {wait_time} detik sebelum mencoba lagi...")
                time.sleep(wait_time)
            else:
                print(f"⚠️ Semua percobaan gagal untuk skor {score}")
                return []

# Loop untuk setiap skor dari 1 sampai 5
print("=" * 60)
print("MEMULAI PROSES SCRAPING DENGAN TARGET 10.000+ ULASAN")
print("=" * 60)

for score in range(1, 6):
    print(f"\n{'='*50}")
    print(f"SCRAPING ULASAN DENGAN SKOR: {score}")
    print(f"{'='*50}")
    
    # Scraping dengan retry mechanism
    reviews_for_score = scrape_reviews_with_retry(app_id, score, target_per_score)
    
    if reviews_for_score:
        # Menambahkan semua ulasan yang didapat untuk skor ini ke list utama
        all_reviews.extend(reviews_for_score)
        print(f"📊 Total ulasan terkumpul sejauh ini: {len(all_reviews)}")
    
    # Memberi jeda 15 detik sebelum lanjut ke skor berikutnya
    print("⏳ Memberi jeda 15 detik...")
    time.sleep(15)

print(f"\n{'='*60}")
print("PROSES SCRAPING SELESAI")
print(f"{'='*60}")
print(f"📊 Total ulasan yang berhasil dikumpulkan: {len(all_reviews)}")

# Validasi apakah sudah mencapai target minimum
if len(all_reviews) >= 10000:
    print("✅ TARGET 10.000+ ULASAN TERCAPAI!")
else:
    print(f"⚠️ Masih kurang {10000 - len(all_reviews)} ulasan dari target 10.000")

MEMULAI PROSES SCRAPING DENGAN TARGET 10.000+ ULASAN

SCRAPING ULASAN DENGAN SKOR: 1
Percobaan ke-1 untuk skor 1
✅ Berhasil mengambil 1670 ulasan untuk skor 1
📊 Total ulasan terkumpul sejauh ini: 1670
⏳ Memberi jeda 15 detik...

SCRAPING ULASAN DENGAN SKOR: 2
Percobaan ke-1 untuk skor 2
✅ Berhasil mengambil 470 ulasan untuk skor 2
📊 Total ulasan terkumpul sejauh ini: 2140
⏳ Memberi jeda 15 detik...

SCRAPING ULASAN DENGAN SKOR: 3
Percobaan ke-1 untuk skor 3
✅ Berhasil mengambil 791 ulasan untuk skor 3
📊 Total ulasan terkumpul sejauh ini: 2931
⏳ Memberi jeda 15 detik...

SCRAPING ULASAN DENGAN SKOR: 4
Percobaan ke-1 untuk skor 4
✅ Berhasil mengambil 1043 ulasan untuk skor 4
📊 Total ulasan terkumpul sejauh ini: 3974
⏳ Memberi jeda 15 detik...

SCRAPING ULASAN DENGAN SKOR: 5
Percobaan ke-1 untuk skor 5
✅ Berhasil mengambil 3000 ulasan untuk skor 5
📊 Total ulasan terkumpul sejauh ini: 6974
⏳ Memberi jeda 15 detik...

PROSES SCRAPING SELESAI
📊 Total ulasan yang berhasil dikumpulkan: 6974
⚠️

### **4. Konversi ke DataFrame dan Eksplorasi Awal**
Setelah semua data terkumpul, mengubahnya menjadi format DataFrame menggunakan Pandas untuk mempermudah analisis dan penyimpanan, melakukan pengecekan awal.

In [4]:
# Mengubah list of dictionaries menjadi DataFrame
df_reviews = pd.DataFrame(all_reviews)

print("\n" + "="*50)
print("ANALISIS DATA HASIL SCRAPING")
print("="*50)

# Menampilkan informasi dasar tentang DataFrame
print("\n📋 Informasi DataFrame:")
df_reviews.info()

print(f"\n📊 Shape data: {df_reviews.shape}")
print(f"📊 Total ulasan: {len(df_reviews)}")

# Mengecek distribusi jumlah ulasan per skor
print(f"\n📈 Distribusi ulasan per skor:")
score_distribution = df_reviews['score'].value_counts().sort_index()
print(score_distribution)

# Mapping rating ke 3 kelas sentimen
def map_rating_to_sentiment(rating):
    """
    Mapping rating 1-5 ke 3 kelas sentimen:
    - Rating 1-2: Negatif
    - Rating 3: Netral  
    - Rating 4-5: Positif
    """
    if rating <= 2:
        return 'negatif'
    elif rating == 3:
        return 'netral'
    else:
        return 'positif'

# Tambahkan kolom sentimen
df_reviews['sentiment'] = df_reviews['score'].apply(map_rating_to_sentiment)

print(f"\n🎯 Distribusi 3 kelas sentimen:")
sentiment_distribution = df_reviews['sentiment'].value_counts()
print(sentiment_distribution)

# Menampilkan 5 baris pertama dari data
print(f"\n📋 5 baris pertama data:")
print(df_reviews[['content', 'score', 'sentiment', 'at']].head())

# Validasi kualitas data
print(f"\n🔍 VALIDASI KUALITAS DATA:")
print(f"- Data kosong pada kolom 'content': {df_reviews['content'].isnull().sum()}")
print(f"- Ulasan dengan content kosong: {df_reviews['content'].str.strip().eq('').sum()}")
print(f"- Rata-rata panjang ulasan: {df_reviews['content'].str.len().mean():.1f} karakter")
print(f"- Ulasan terpendek: {df_reviews['content'].str.len().min()} karakter")
print(f"- Ulasan terpanjang: {df_reviews['content'].str.len().max()} karakter")


ANALISIS DATA HASIL SCRAPING

📋 Informasi DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6974 entries, 0 to 6973
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              6974 non-null   object        
 1   userName              6974 non-null   object        
 2   userImage             6974 non-null   object        
 3   content               6974 non-null   object        
 4   score                 6974 non-null   int64         
 5   thumbsUpCount         6974 non-null   int64         
 6   reviewCreatedVersion  5335 non-null   object        
 7   at                    6974 non-null   datetime64[ns]
 8   replyContent          6311 non-null   object        
 9   repliedAt             6311 non-null   datetime64[ns]
 10  appVersion            5335 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 599.5+ KB

📊 Shape data:

### **5. Penyimpanan Dataset**
Langkah terakhir adalah menyimpan DataFrame yang berisi data bersih ke dalam sebuah file `.csv`. File ini akan digunakan pada notebook selanjutnya untuk tahap preprocessing dan pemodelan.

In [5]:
# Simpan dengan informasi timestamp dan validasi
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f'dataset_mamikos_scraped_{current_time}.csv'

# Pilih kolom yang diperlukan untuk analisis sentimen
columns_to_save = [
    'reviewId', 'userName', 'content', 'score', 'sentiment', 
    'thumbsUpCount', 'at', 'replyContent'
]

# Menyimpan DataFrame ke file CSV
df_reviews[columns_to_save].to_csv(output_filename, index=False)

print(f"\n💾 Dataset berhasil disimpan sebagai '{output_filename}'")
print(f"📁 Kolom yang disimpan: {columns_to_save}")

# Tampilkan ringkasan akhir untuk validasi
print(f"\n{'='*60}")
print("RINGKASAN HASIL SCRAPING")
print(f"{'='*60}")
print(f"✅ Total data: {len(df_reviews):,} ulasan")
print(f"✅ Target minimum (10.000): {'TERCAPAI' if len(df_reviews) >= 10000 else 'BELUM TERCAPAI'}")
print(f"✅ 3 kelas sentimen: {list(sentiment_distribution.index)}")
print(f"✅ Distribusi sentimen:")
for sentiment, count in sentiment_distribution.items():
    percentage = (count / len(df_reviews)) * 100
    print(f"   - {sentiment.capitalize()}: {count:,} ({percentage:.1f}%)")

print(f"\n🎉 Bagian 1 (Scraping) SELESAI!")
print(f"📝 Lanjut ke Bagian 2: Preprocessing & Training Model")

# Simpan juga file backup dalam format yang berbeda
backup_filename = f'dataset_mamikos_backup_{current_time}.json'
df_reviews[columns_to_save].to_json(backup_filename, orient='records', lines=True)
print(f"💾 Backup data disimpan sebagai '{backup_filename}'")


💾 Dataset berhasil disimpan sebagai 'dataset_mamikos_scraped_20250609_220446.csv'
📁 Kolom yang disimpan: ['reviewId', 'userName', 'content', 'score', 'sentiment', 'thumbsUpCount', 'at', 'replyContent']

RINGKASAN HASIL SCRAPING
✅ Total data: 6,974 ulasan
✅ Target minimum (10.000): BELUM TERCAPAI
✅ 3 kelas sentimen: ['positif', 'negatif', 'netral']
✅ Distribusi sentimen:
   - Positif: 4,043 (58.0%)
   - Negatif: 2,140 (30.7%)
   - Netral: 791 (11.3%)

🎉 Bagian 1 (Scraping) SELESAI!
📝 Lanjut ke Bagian 2: Preprocessing & Training Model
💾 Backup data disimpan sebagai 'dataset_mamikos_backup_20250609_220446.json'
